In [1]:
from dask.distributed import Client, LocalCluster
if __name__ == "__main__":
    cluster=LocalCluster(host="tcp://127.0.0.1:37790",dashboard_address="127.0.0.1:8787",n_workers=4)
    client = Client(cluster)

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import math
import dask

import skimage.feature
import skimage.segmentation
import scipy.ndimage as ndi

import glob

import multiprocessing
from multiprocessing import Pool, Manager

In [2]:
def pre_process(U,V):
    try:
        U.longitude
        U.latitude
    except:
        V = V.rename({'lon':'longitude'})
        V = V.rename({'lat':'latitude'})
        U = U.rename({'lon':'longitude'})
        U = U.rename({'lat':'latitude'})
        U.longitude
        U.latitude
    try:
        test = U.uas
    except:
        try:
            test = U.ua
            U = U.rename({'ua':'uas'})
            V = V.rename({'va':'vas'})
        except:
            U = U.rename({'uwnd':'uas'})
            V = V.rename({'vwnd':'vas'})
    try:
        test = U.level
    except:
        try:
            test = U.plev
            U = U.rename({'plev':'level'})
            V = V.rename({'plev':'level'})
        except:
            U = U.rename({'height':'level'})
            V = V.rename({'height':'level'})
    return U,V

In [157]:
def front_watershed(U,V,lon,lat):
    print('front_watershed')
    front = U.copy()
    front['uas'] = front.uas*0
    front = front.rename({'uas':'x'})
    front['U2'] = U.uas #now
    front['V2'] = V.vas #now
    front['U1'] = xr.DataArray(np.concatenate([U.uas.values[:2]*np.nan,U.uas.values[:-2]]),dims=("time","latitude", "longitude"), coords={"time":front.time.values,"longitude":lon ,"latitude": lat})
    front['V1'] = xr.DataArray(np.concatenate([V.vas.values[:2]*np.nan,V.vas.values[:-2]]),dims=("time","latitude", "longitude"), coords={"time":front.time.values,"longitude":lon ,"latitude": lat})
    front = front.sel(time=front.time[2:])
    x = xr.where(front['U1'].values>0,front['U1'].values/front['U1'].values,np.nan)
    x = xr.where(front['V1'].values<0,x,np.nan)
    x = xr.where(front['U2'].values>0,x,np.nan)
    x = xr.where(front['V2'].values>0,x,np.nan)
    x = xr.where(front['V2'].values - front['V1'].values >  2.0 ,x,np.nan)
    front['x'] = xr.DataArray(x,dims=("time","latitude", "longitude"), coords={"time":front.time.values,"longitude":lon ,"latitude": lat}) #were above frontal criteria are met
    front['front'] = xr.DataArray(x*np.nan,dims=("time","latitude", "longitude"), coords={"time":front.time.values,"longitude":lon ,"latitude": lat}) #empty for output
    x = np.nan_to_num(x).astype('int64')
    for d in range(2,len(front.time)): #watershed segmentation to label each front
        front_idx = skimage.feature.peak_local_max(x[d], footprint=np.ones((3, 3)),labels=x[d])
        front_mask = np.full_like(x[d], fill_value = False)
        front_mask[tuple(front_idx.T)] = True
        markers = ndi.label(front_mask)[0]
        obj = skimage.segmentation.watershed(-x[d], markers, mask=front_mask)
        z = xr.DataArray(obj, dims=("lat", "lon"), coords={"lon": lon , "lat":lat})
        front['front'][d] = z
    front['front'] = front.front/front.front
    return front

In [5]:
def getfront(frontin,dx='dx',dy='dy',lat='lat',lon='lon'):
    print('getfront')
    front = frontin
    front = front.sel(time=front.time[1:])
    for d in range(len(front.time)):
        winners=[]
        if math.isnan(np.nanmax(front.front[d]))==False:
            for z in range(1,int(np.nanmax(front.front[d]))+1):
                ids =  np.argwhere(front.front[d].values==z)
                latsid = [item[0] for item in ids]
                lonsid = [item[1] for item in ids]
                lats = [lat[i] for i in latsid]
                lons = [lon[i] for i in lonsid]
                mlat = np.mean(lats)
                imlat = np.argmin((lat-mlat)**2) #mean lat
                #area = dy*np.sum(dx[latsi[z]]) #area not sure why though not used again?
                xlen = dx[imlat]*4*np.std(lons)
                ylen = dy*4*np.std(lats)
                length = np.sqrt((ylen**2)+(xlen**2))  #length
                if length>500.0:   #must be greater than 500km
                    winners.append(z)
            front['front'][d] = front.front[d].where(np.isin(front.front[d],winners))
    return front


In [3]:
ua = glob.glob("/terra/data/reanalysis/global/reanalysis/ECMWF/ERA5/6hr/native/ua*")
va = glob.glob("/terra/data/reanalysis/global/reanalysis/ECMWF/ERA5/6hr/native/va*")

In [26]:
U = xr.open_mfdataset(ua)
V = xr.open_mfdataset(va)

In [27]:
U_full,V_full = pre_process(U,V)

In [33]:
for year in range(1979,2021):
    print(year)
    U = U_full.sel(time=str(year)) 
    V = V_full.sel(time=str(year))
    U = U.sel(level=950) 
    V = V.sel(level=950) 
    U = U.sel(latitude = slice(-15,-75))
    U = xr.concat([U.sel(longitude = slice(0,30)),U.sel(longitude = slice(320,360))],dim='longitude')
    V = V.sel(latitude = slice(-15,-75))
    V = xr.concat([V.sel(longitude = slice(0,30)),V.sel(longitude = slice(320,360))],dim='longitude')
    V.coords['longitude'] = (V.coords['longitude'] + 180) % 360 - 180
    U.coords['longitude'] = (U.coords['longitude'] + 180) % 360 - 180
    U = U.sortby(U.longitude).load()
    V = V.sortby(U.longitude).load()
    lon = U.longitude.values
    nlon = len(lon)
    lat = U.latitude.values
    nlat = len(lat)
    front = front_watershed(U,V,lon,lat)
    U.close()
    V.close()
    front = front[['front']]
    lon = U.longitude.values
    nlon = len(lon)
    lat = U.latitude.values
    nlat = len(lat)
    dx = np.cos(lat*math.pi/180.0)*2*math.pi*6370/360*(lon[-1]-lon[1])/(nlon-1)
    dy = ((lat[2]-lat[1])/180.0)*6370*math.pi
    front = getfront(front,dx,dy,lat,lon)
    front = front.front.fillna(0)
    front.to_netcdf(path='../FRONT_FILES/native/3-hourly/ERA5_6hr/ERA5_front_native_'+str(year)+'.nc', mode='w')
    front.close()

1979
front_watershed
getfront


<ipython-input-5-f9f28d8ed393>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


1980
front_watershed
getfront


<ipython-input-5-f9f28d8ed393>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


1981
front_watershed
getfront


...

<ipython-input-5-f9f28d8ed393>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


2019
front_watershed
getfront


<ipython-input-5-f9f28d8ed393>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


2020
front_watershed
getfront


<ipython-input-5-f9f28d8ed393>:7: RuntimeWarning: All-NaN axis encountered
  if math.isnan(np.nanmax(front.front[d]))==False:


In [ ]:
#CDO mergetime used to concatenate times

In [ ]:
front = xr.open_mfdataset(glob.glob("/home/pmarsh/MASTERS/FRONT/FRONT_FILES/native/3-hourly/ERA5_6hr/*"))

front_seas = front.front.groupby(front.time.dt.season).mean(dim='time')

front_seas.to_netcdf(path='/home/pmarsh/MASTERS/FRONT/FRONT_FILES/native/ERA5_seasonal_mean_front_density_native.nc', mode='w')

In [14]:
client.close()